In [2]:
import theano
import theano.tensor as T
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K2200


In [51]:
scores = T.matrix('scores')
# Images
diag   = T.diag(scores)
diff_img = scores - diag.dimshuffle(0, 'x') + 1
max_img = T.maximum(0, diff_img)
triu_img = T.triu(max_img, 1)
til_img  = T.tril(max_img, -1)
res_img = T.sum(triu_img) + T.sum(til_img)
# Sentences
diff_sent = scores.T - diag.dimshuffle(0, 'x') + 1
max_sent = T.maximum(0, diff_sent)
triu_sent = T.triu(max_sent, 1)
til_sent  = T.tril(max_sent, -1)
res_sent = T.sum(triu_sent) + T.sum(til_sent)

f = theano.function([scores], res_img+res_sent)

In [52]:
%%time
m = [[10,2,3], [4,10,6], [7,9,15]]

print np.array(m)
print f(m)

[[10  2  3]
 [ 4 10  6]
 [ 7  9 15]]
0.00999999977648
CPU times: user 7 ms, sys: 0 ns, total: 7 ms
Wall time: 5.8 ms


In [13]:
def loss_loop(k, num_imgs, scores):
    inner_loop_idxs = T.neq(T.arange(scores.shape[0]), k).nonzero()[0]
    inner_loop_idxs.name = 'inner_loop_idxs'

    results_imgs, _ = theano.scan(
        fn=lambda l, scores: T.maximum(0, scores[k,l] - scores[k,k] + 1),
        sequences=[inner_loop_idxs],
        non_sequences=[scores]
    )
    results_sents, _ = theano.scan(
        fn=lambda l, scores: T.maximum(0, scores[k,l] - scores[k,k] + 1),
        sequences=[inner_loop_idxs],
        non_sequences=[scores.T]
    )
    results_imgs = T.sum(results_imgs)
    results_sents = T.sum(results_sents)

    return results_imgs + results_sents

scores = T.matrix('scores')
num_imgs = scores.shape[0]
num_imgs.name = 'num_imgs'
loss_loop_idxs = T.arange(scores.shape[0])
loss_loop_idxs.name = 'loss_loop_idxs'

results, _ = theano.scan(
    fn=loss_loop,
    outputs_info=None,
    sequences=[loss_loop_idxs],
    non_sequences=[num_imgs, scores]
)

f = theano.function([scores], T.sum(results))

MissingInputError: A variable that is an input to the graph was neither provided as an input to the function nor given a value. A chain of variables leading from this input to an output is [loss_loop_idxs[t], for{cpu,scan_fn}.0]. This chain may not be unique
Backtrace when the variable is created:
  File "<ipython-input-13-c59e0271f779>", line 25, in <module>
    non_sequences=[num_imgs, scores]


In [10]:
%%time
f([[1,2],[3,4]])

CPU times: user 2 ms, sys: 4 ms, total: 6 ms
Wall time: 6.43 ms


array(5.0, dtype=float32)

In [4]:
ifrags = T.matrix('ifrags')
sfrags = T.matrix('sfrags')
scores = T.matrix('scores')

result, updates = theano.scan(fn=lambda sfrag, idx, ifrags, scores: scores,
                             sequences=[sfrags, T.arange(100)],
                             non_sequences=[ifrags, scores])

final = theano.function([ifrags, sfrags, scores], result)

In [3]:
ifrags = np.random.rand(20,600).astype(np.float32)
sfrags = np.random.rand(9,1,600).astype(np.float32)
scores = np.zeros((ifrags.shape[0], sfrags.shape[0]), dtype=np.float32)
final(ifrags, sfrags, scores)

TypeError: ('Bad input argument to theano function with name "<ipython-input-2-47aa487bb2cf>:9"  at index 1(0-based)', 'Wrong number of dimensions: expected 2, got 3 with shape (9, 1, 600).')

In [5]:
x = T.matrix('x')
y = T.tensor3('y')
output = T.max(T.batched_dot(T.shape_padleft(x, 1).repeat(T.shape(y)[0], 0), y.dimshuffle(0, 2, 1)), axis=1)
f = theano.function([x, y], output)

In [6]:
ifrags = np.random.rand(20, 600).astype(np.float32)
sfrags = np.random.rand(9, 1, 600).astype(np.float32)
f(ifrags, sfrags)

array([[ 151.43132019],
       [ 152.62065125],
       [ 161.51893616],
       [ 160.23997498],
       [ 163.73124695],
       [ 158.63717651],
       [ 157.13568115],
       [ 159.2283783 ],
       [ 154.00354004]], dtype=float32)

In [12]:
import sys
sys.path.insert(0, '/home/flipvanrijn/Workspace/Dedicon-Thesis/')

from keras.models import Sequential

sent_model = Sequential()
sent_model.add(BiDirection(10, 300, 600, activation='relu', output_mode='concat'))

NameError: name 'BiDirection' is not defined

In [ ]:
sent_model.get_output()

In [ ]:
import theano
f_combined = theano.function([sent_model.get_input()], sent_model.get_output())

In [ ]:
import numpy as np
f_combined(np.ones((1,9, 10), dtype=np.float32))